# Group ML income predict

In [0]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [0]:
ankitaTrain = pd.read_csv('tcd-ml-1920-group-income-train.csv')
ankitaTest = pd.read_csv('tcd-ml-1920-group-income-test.csv')    

U:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
U:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
ankitaTrain.head()

,Instance,Year of Record,Housing Situation,Crime Level in the City of Employement,Work Experience in Current Job [years],Satisfation with employer,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Yearly Income in addition to Salary (e.g. Rental Income),Total Yearly Income [EUR]
0,1,1940.0,0,33,17,Unhappy,other,45,Afghanistan,25179,group head,No,1,Black,182,0 EUR,6182.05
1,2,1940.0,0,25,4.9,Unhappy,female,17,Afghanistan,2278204,heavy vehicle and mobile equipment service tec...,No,0,Blond,172,0 EUR,6819.69
2,3,1940.0,0,34,21,Unhappy,female,48,Afghanistan,822134,sorter,Bachelor,0,Blond,144,0 EUR,8663.53
3,4,1940.0,0,70,18,Average,female,42,Albania,59477,quality control senior engineer,No,1,Brown,152,0 EUR,2400.64
4,5,1940.0,0,51,8,Happy,other,15,Albania,23494,logistician,Master,1,Black,180,0 EUR,2816.18


In [0]:
ankitaTest.head()

,Instance,Year of Record,Housing Situation,Crime Level in the City of Employement,Work Experience in Current Job [years],Satisfation with employer,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Yearly Income in addition to Salary (e.g. Rental Income),Total Yearly Income [EUR]
0,1,1994.0,Small House,23,12,Average,other,23,Serbia,734369,quality assurance specialist,Bachelor,0,Brown,151,0 EUR,NaN
1,2,1964.0,Castle,16,20,Average,NaN,44,Austria,897352,student data analyst,Master,0,Black,181,0 EUR,NaN
2,3,1974.0,Large House,22,11,Average,NaN,21,Serbia,766,project manager,Bachelor,0,Blond,179,0 EUR,NaN
3,4,1997.0,Large Apartment,41,14,Average,male,24,Sierra Leone,1150488,staff engineer / architect,Bachelor,0,Blond,201,0 EUR,NaN
4,5,1949.0,0,22,17,Happy,male,42,Ecuador,98532,machinist,No,1,Blond,163,0 EUR,NaN


In [0]:
ankitaTrain[ankitaTrain.columns]=ankitaTrain[ankitaTrain.columns].fillna(ankitaTrain.mode().iloc[0])
ankitaTest[ankitaTest.columns]=ankitaTest[ankitaTest.columns].fillna(ankitaTest.mode().iloc[0])

# Pre Processing of Data

In [0]:
ankitaTrain['Gender'].value_counts()

In [0]:
ankitaTrain["Gender"] = ankitaTrain["Gender"].replace('f','female')
ankitaTrain["Gender"] = ankitaTrain["Gender"].replace('0','unknown')

In [0]:
ankitaTrain['Gender'].value_counts()

male       474240
female     252520
other      252053
unknown     69761
Name: Gender, dtype: int64

In [0]:
ankitaTest['Gender'].value_counts()

male       167280
other       88688
female      83235
unknown     22441
f            5456
0            2338
Name: Gender, dtype: int64

In [0]:
ankitaTest["Gender"] = ankitaTest["Gender"].replace('f','female')
ankitaTest["Gender"] = ankitaTest["Gender"].replace('0','unknown')

In [0]:
a = statistics.mode(ankitaTrain['Housing Situation'])
ankitaTrain['Housing Situation'] = ankitaTrain['Housing Situation'].replace('nA',a)
ankitaTrain['Housing Situation'] = ankitaTrain['Housing Situation'].astype(str)

In [0]:
a = statistics.mode(ankitaTest['Housing Situation'])
ankitaTest['Housing Situation'] = ankitaTest['Housing Situation'].replace('nA',a)
ankitaTest['Housing Situation'] = ankitaTest['Housing Situation'].astype(str)

In [0]:
ankitaTest['University Degree'] = ankitaTest['University Degree'].replace('0','No')
ankitaTrain['University Degree'] = ankitaTrain['University Degree'].replace('0','No')

In [0]:
ankitaTrain['Work Experience in Current Job [years]'] = ankitaTrain['Work Experience in Current Job [years]'].replace('#NUM!',ankitaTrain['Work Experience in Current Job [years]'].mode())
ankitaTest['Work Experience in Current Job [years]'] = ankitaTest['Work Experience in Current Job [years]'].replace('#NUM!',ankitaTest['Work Experience in Current Job [years]'].mode())

In [0]:
data = pd.concat([ankitaTrain, ankitaTest], ignore_index=True)

In [0]:
data['Yearly Income in addition to Salary (e.g. Rental Income)'] = data['Yearly Income in addition to Salary (e.g. Rental Income)'].str.replace('([A-Za-z]+)', '')
data['Yearly Income in addition to Salary (e.g. Rental Income)'] = pd.to_numeric(data['Yearly Income in addition to Salary (e.g. Rental Income)'])

In [1]:
data['Work Experience in Current Job [years]'] = pd.to_numeric(data['Work Experience in Current Job [years]'])

NameError: ignored

In [0]:
data.dtypes

Instance                                                      int64
Year of Record                                              float64
Housing Situation                                           float64
Crime Level in the City of Employement                        int64
Work Experience in Current Job [years]                      float64
Satisfation with employer                                   float64
Gender                                                      float64
Age                                                           int64
Country                                                     float64
Size of City                                                  int64
Profession                                                  float64
University Degree                                           float64
Wears Glasses                                                 int64
Hair Color                                                  float64
Body Height [cm]                                

In [0]:
for i in data.dtypes[data.dtypes == 'object'].index.tolist():
    grp_hs = data.groupby(i)["Total Yearly Income [EUR]"].agg("mean")
#     data[i] = featureLE.transform(data[i].astype(str))
    data[i] = data[i].map(grp_hs) 

In [0]:
data.head()

,Instance,Year of Record,Housing Situation,Crime Level in the City of Employement,Work Experience in Current Job [years],Satisfation with employer,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Yearly Income in addition to Salary (e.g. Rental Income),Total Yearly Income [EUR]
0,1,1940.0,8842.45964,33,56865.123725,157393.908986,71399.961738,45,191828.953279,25179,55301.387315,60670.032070,1,68615.023508,182,0.0,6182.05
1,2,1940.0,8842.45964,25,38186.601367,157393.908986,65240.279550,17,191828.953279,2278204,62616.405270,60670.032070,0,68632.591771,172,0.0,6819.69
2,3,1940.0,8842.45964,34,61991.120391,157393.908986,65240.279550,48,191828.953279,822134,70670.102123,65938.885726,0,68632.591771,144,0.0,8663.53
3,4,1940.0,8842.45964,70,57817.113752,65860.953181,65240.279550,42,135701.550567,59477,63171.291534,60670.032070,1,68029.358690,152,0.0,2400.64
4,5,1940.0,8842.45964,51,43420.235712,68667.004068,71399.961738,15,135701.550567,23494,66590.907652,76785.113519,1,68615.023508,180,0.0,2816.18


In [0]:
data.columns

Index(['Instance', 'Year of Record', 'Housing Situation',
       'Crime Level in the City of Employement',
       'Work Experience in Current Job [years]', 'Satisfation with employer',
       'Gender', 'Age', 'Country', 'Size of City', 'Profession',
       'University Degree', 'Wears Glasses', 'Hair Color', 'Body Height [cm]',
       'Yearly Income in addition to Salary (e.g. Rental Income)',
       'Total Yearly Income [EUR]'],
      dtype='object')

In [0]:
data.shape

(1418012, 17)

In [0]:
ankitaTrain.shape

(1048574, 17)

In [0]:
ankitaTest.shape

(369438, 17)

In [0]:
Training_X,Testing_X = data[data.columns].iloc[:1048573],data[data.columns].iloc[1048574:]

In [0]:
Training_Y = data['Total Yearly Income [EUR]'].iloc[:1048573]

In [0]:
Training_X.drop(['Instance', 'Total Yearly Income [EUR]'], axis=1 , inplace=True)
Testing_X.drop(['Instance', 'Total Yearly Income [EUR]'], axis=1, inplace=True)

In [0]:
print(Training_X.shape)
print(Testing_X.shape)
print(Training_Y.shape)

(1048573, 15)
(369438, 15)
(1048573,)


In [0]:
reg = LinearRegression().fit(Training_X, Training_Y)

In [0]:
Testing_X[Testing_X.columns]=Testing_X[Testing_X.columns].fillna(Testing_X.mode().iloc[0])

In [0]:
Training_X,xValidation,Training_Y,yValidation = train_test_split(Training_X,Training_Y,test_size=0.2,random_state=1234)    

In [0]:
reg = LinearRegression().fit(Training_X, Training_Y)

In [0]:
Predictions_Y = reg.predict(Testing_X)

In [0]:
ak = pd.read_csv('tcd-ml-1920-group-income-submission.csv')

In [0]:
ak['Total Yearly Income [EUR]'] = Predictions_Y

In [0]:
ak

,Instance,Total Yearly Income [EUR]
0,1,47973.752820
1,2,23137.275086
2,3,-17407.746221
3,4,130703.997424
4,5,48229.721636
...,...,...
369433,369434,81928.071278
369434,369435,299871.582887
369435,369436,44260.223369
369436,369437,161037.031233


In [0]:
ak.to_csv('ak.csv',index=False)

# Applying Model

In [0]:
model=CatBoostRegressor(iterations=2000, depth=10, learning_rate=0.01, loss_function='MAE')
model.fit(Training_X,Training_Y,cat_features=None,eval_set=(xValidation, yValidation))
y_predi = model.predict(xValidation)

0:	learn: 68189.1765920	test: 67869.4747052	best: 67869.4747052 (0)	total: 373ms	remaining: 12m 25s
1:	learn: 67685.0483556	test: 67368.7484442	best: 67368.7484442 (1)	total: 610ms	remaining: 10m 9s
2:	learn: 67158.5633240	test: 66845.8673513	best: 66845.8673513 (2)	total: 839ms	remaining: 9m 18s
3:	learn: 66948.0880050	test: 66635.4014874	best: 66635.4014874 (3)	total: 1.02s	remaining: 8m 30s
4:	learn: 66691.9821324	test: 66379.5870198	best: 66379.5870198 (4)	total: 1.24s	remaining: 8m 13s
5:	learn: 66384.8830704	test: 66073.6776157	best: 66073.6776157 (5)	total: 1.46s	remaining: 8m 4s
6:	learn: 66095.7905640	test: 65785.5201712	best: 65785.5201712 (6)	total: 1.68s	remaining: 7m 57s
7:	learn: 65810.7599768	test: 65502.0265221	best: 65502.0265221 (7)	total: 1.9s	remaining: 7m 54s
8:	learn: 65315.6881623	test: 65010.8240630	best: 65010.8240630 (8)	total: 2.11s	remaining: 7m 47s
9:	learn: 64841.5351885	test: 64540.3101040	best: 64540.3101040 (9)	total: 2.34s	remaining: 7m 46s
10:	learn: 

# MAE Calculation

In [0]:
y_predi = model.predict(xValidation)
mean_absolute_error(yValidation, y_predi)

9806.318818343085

# Predict and Save File

In [0]:
Predictions_Y = model.predict(Testing_X)
ak = pd.read_csv('tcd-ml-1920-group-income-submission.csv')
ak['Total Yearly Income [EUR]'] = Predictions_Y
ak.to_csv('MLSub.csv',index=False)

In [0]:
ak

,Instance,Total Yearly Income [EUR]
0,1,48060.998722
1,2,23228.217376
2,3,-17319.250294
3,4,130706.649618
4,5,48244.968652
...,...,...
369433,369434,81886.259247
369434,369435,299327.862247
369435,369436,44211.726894
369436,369437,160838.478171
